In [ ]:
# Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Detecting Changes in Sentinel-1 Imagery (Part 4)

Author: mortcanty

In the fourth and final part of this Community Tutorial series on SAR change detection, we will have a look at some more examples using imagery taken from the GEE Sentinel-1 Archive. In order to encourage further exploration,  the algorithm that we developed in the first three parts has been provided with a convenient interactive widget interface. Some of the examples presented here are taken from the publications 
[Canty et al. (2020)](https://www.mdpi.com/2072-4292/12/1/46) and [Canty et al. (2020a)](https://www.mdpi.com/2072-4292/12/15/2454).




### Preliminaries

In [ ]:
# Setup the GEE Python API
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=C1tv4R_bu9NHApqwNCMBsxfGssCmj_gt2cAE85nPezc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhr1XmvxXHUoe0FgkRfwIJLS4v4EwW0OZfQNMh4p9Wrhue5yuG4BwE

Successfully saved authorization token.


In [ ]:
# Widget manager
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Install the algorithm and the interactive interface (from fresh runtime!)
!pip install -q ipyleaflet
!git clone https://github.com/mortcanty/eesarseq
%cd /content/eesarseq/src
%run setup install

     |████████████████████████████████| 3.3 MB 5.1 MB/s 
Cloning into 'eesarseq'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 304 (delta 165), reused 244 (delta 105), pack-reused 0
Receiving objects: 100% (304/304), 11.28 MiB | 34.78 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/eesarseq/src
running install
running build
running build_py
running install_lib
creating /usr/local/lib/python3.7/dist-packages/eesar
copying build/lib/eesar/application.py -> /usr/local/lib/python3.7/dist-packages/eesar
copying build/lib/eesar/__init__.py -> /usr/local/lib/python3.7/dist-packages/eesar
byte-compiling /usr/local/lib/python3.7/dist-packages/eesar/application.py to application.cpython-37.pyc
byte-compiling /usr/local/lib/python3.7/dist-packages/eesar/__init__.py to __init__.cpython-37.pyc
running install_egg_info
Writing /usr/local/lib/python3.7/dist-packages/eesar-1.0.eg

# Part 4. Applications: Exploring the GEE Sentinel-1 Collection



In [ ]:
# Run the interface
from eesar.application import run
run()  

Map(center=[51.0, 6.4], controls=(DrawControl(options=['position'], polygon={'shapeOptions': {'fillColor': '#0…

## How to Use the Interface

### Caveat

The sequential SAR change detection method that we developed in the first three parts of this tutorial is _pixel-oriented_. That is to say, it is based entirely on the statistical properties of each and every multi-look pixel in the observed image series. For this reason the method is not a particularly good fit to the GEE paradigm: Upscaling to very large geographical areas is not practical. Furthermore, since we are analysing time series for changes, the reflectance images involved must all completely overlap the region of interest and have been irradiated from the same position in space. This means choosing either ascending **or** descending node, and only **one** relative orbit number for any given sequence.

### Walk through

To get started, let's see how to generate a small change map. In the widget interface above, choose **Platform** A, leaving all other settings as is. Select a small area of interest (aoi) near the town of Jülich with the rectangular or polygon **draw tool**. This will enable the **Collect** button. Click it to collect an image series, the details of which are printed in the info box at the bottom. The raster overlay shows the complete swath of the last image in the sequence. When the overlay is fully rendered, the **Preview** button is enabled. 

**Note:** Depending on the position of the aoi, two relative orbit numbers may be displayed (88 and 15). If so, in the corresponding **RelOrbit** field choose 88 and re-collect. 

The **Preview** button will  now trigger the change detection algorithm at the scale selected by the current zoom setting. The color coded change map is displayed, showing the index of the **First** detected change for each pixel, corresponding to the Radio Button selection next to **Preview** (palette = 'black, blue, cyan, yellow, red' indicating early in the series through to late).  Since processing is carried out in parallel, the change image is built up tile-by-tile. As explained in Part 3 of this tutorial, the zoom setting can falsify the result somewhat, depending on the pyramid level at which the calculation is carried out. Nevertheless it is often convenient for generating a quick overview. You can see the effect by zooming in and out. De-select the **QuickPreview** check box to override it. Now the calculation is carried out at the full 10m pixel scale irrespective of the zoom level chosen, but can take considerably longer.

If and when you are satisfied with the previewed result, you can export the change maps to your GEE cloud assets with the **ExportToAssets** button. 

### The widgets

**Platform:** Choose one or both of the Sentinel-1 satellites.

**Pass:** Choose ascending or descending node.

**RelOrbit:** Choose relative orbit number. If set to 0 all images are included with orbit numbers which overlap with the area of interest.

**StartDate:** Beginning of collected time series.

**EndDate:** End of collected time series.

**Collect:** Start collection, enabled when an area of interest has been chosen. Upon completion the last Sentinel-1 image in the sequence is displayed.

**Signif:** Choose a significance level for the likelihood ratio test

**MedianFilter:** Run a 5x5 median filter over the change map before displaying or exporting

**Stride:** Select only a subset of the collected sequence

**ShowS2:** Display the most cloud-free Sentinel-2 image found within the chosen time period instead of the last Sentinel-1 image

**ExportToAssets:** Creates and runs a batch task to export a change map Image as a raster to an Earth Engine asset.  For a time series of $k$ images, the exported change map consists of $k+2$ bands
- cmap: the interval* of the most recent change, one band, byte values $\in [0,k-1]$, where 0 = no change.
- smap: the interval of the first change, one band, byte values $\in [0,k-1]$, where 0 = no change.
- fmap: the number of changes, one band, byte values $\in [0,k-1]$, where 0 = no change.
- bmap: the changes in each interval, $\ k-1$ bands, byte values $\in [0,3]$, where 0 = no change, 1 = positive definite change, 2 = negative definite change, 3 = indefinite change.

*Two successive acquisition times in the series.

**ExportToDrive:** Sends the change map described above to Drive storage in GeoTiff format.

**Preview:** Run the change detection algorithm and preview results according to the chosen settings (often slow, depending upon series length, zoom level and size of the aoi).

**ReviewAsset:** Display a currently selected change map asset according to the chosen settings (very fast, since calculations have already been performed).

**PlotFromAsset:** Plot the proportion of change pixels in the bmap bands of the selected asset as a function of time.

**Bitemporal:** Preview (or ReviewFrom Asset) the change map for one interval on the series (black = no change, red = positive definite, cyan = negative definite, yellow = indefinite). 

**Interval:** Choose the interval for the Bitemporal map.

**First:** Preview (or ReviewFromAsset) the smap band (palette = 'black, blue, cyan, yellow, red' indicating no change or early in the series through to late). 

**Last:** Preview (or ReviewFromAsset) the cmap band (palette = 'black, blue, cyan, yellow, red' indicating no change or early in the series through to late). 

**Frequency:**

**MaxFreq:**

**NCMask:**

**WaterMask:**

**QuickPreview:**

**Opacity:**

**Clear:**

**GoTo:**

## Some Examples

### Surveillance

Remote sensing surveillance, or monitoring, applications are most often associated with very high resolution satellite or aereal sensors. The Sentinel-1 C-band ground range detected imagery on GEE is obviously much less applicable to such tasks. This is mainly due to the relatively poor spatial resolution ($\approx 20m$) and the ever-present speckle in the archived images. 

On the other hand, a guaranteed, all-weather revisit time of 6 days might be usefully exploited in some circumstances. Seaport activity monitoring is one possibility, with the frequency of large vessel arrival and departure (usually quoted as ATT or average turnaround time) being in general of the order of days. Thus a change signal generated every 6 days from our sequential algorithm might be expected to catch a large proportion of cargo or large passenger ship movements at a given seaport. 

As a specific example, ports along the coast of Libya have been the subject of recent international attention, in part due to the refugee crisis in the Mediterranean and also because of Libyan political instability. Some of them have been closed periodically to international traffic. The port of Benghazi was closed in the first part of 2017, reopening on 5 October. This is verified in the change frequency plots below which were generated directly from the interface. The change observations are partly confused by noise from open water, caused mostly by surface wave conditions, but the signals of interest are quite clear. (The water mask would remove the noise, but also hide some ship movements.)

<img src="https://github.com/mortcanty/eesarseq/blob/main/screenshots/bengasi2017.png?raw=1"><img/>

Fig. 1. Port of Benghasi change frequency 2017-01-01 to 2017-10-04 (44 images, platforms A and B, relative orbit 73, ascending node).

<img src="https://github.com/mortcanty/eesarseq/blob/main/screenshots/bengasi2018.png?raw=1"><img/>

Fig. 2. As Figure 1 for 2018-01-01 to 2018-10-04.
    
<img src="https://github.com/mortcanty/eesarseq/blob/main/screenshots/bengasi2017-8.png?raw=1"><img/>

Fig. 3. Proportion of changed pixels in a small aoi around the main dock in Benghasi for a 60-image time sequence. There were about 900  pixels in all in the aoi.

The covid pandemic had a very significant effect on Mediterranean tourist seaports and, unless you happen to be maybe a Venitian merchant or hotel operator, certainly a positive one. Try using the same techniques as above on the main passenger ship dock in Venice, choosing a time series covering pre-covid to the present.

Another, more future-oriented application of SAR change detection, might be the verification of the de-commissioning of large open face coal mines in national programs for $CO_2 $ reduction. The Hambach open pit coal mine west of Cologne, Germany is depicted in the single interval, Loewner order change image below. The large digging and back-filling machines are visible in the change series only by virtue of their movement (cyan: machine present $\to$ machine absent, and red: vice versa) and would disappear if all mining activity were to stop.

<img src="https://github.com/mortcanty/eesarseq/blob/main/screenshots/hambach2020.png?raw=1"><img/>

Fig. 3. Single interval (bitemporal) changes (May 13 to May 19) over the Hambach open pit mine from a series of 55 images covering the year 2020. 


### Inundation

In [Part 3](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3) of this series we used a catastophic flooding event in England to introduce and illustrate the sequential change detection algorithm. This is in fact a widely known application of SAR.  Quoting from the [Office for Outer Space Affairs
UN-SPIDER Knowledge Portal](https://un-spider.org/advisory-support/recommended-practices/recommended-practice-flood-mapping/python-step-by-step),

_The usage of Synthetic Aperture Radar (SAR) satellite imagery for flood extent mapping constitutes a viable solution with fast image processing, providing **near real-time flood information** (my emphasis) to relief agencies for supporting humanitarian action. The high data reliability as well as the absence of geographical constraints,  such as site accessibility, emphasize the technology’s potential in the field._

And here, as another illustration, is an excerpt from [Canty et al. (2020)](https://www.mdpi.com/2072-4292/12/1/46):

_Cyclone Idai, recorded as the worst weather-based event to ever occur in the southern hemisphere, made landfall near the city of Beira, Mozambique on March 15, 2019 causing widespread inundation in Mozambique and Tanzania and a death toll of more than 1300. Figure 4 shows a sequence of six change maps generated with a GEE time series of Sentinel-1a and Sentinel-1b images. The reduction of reflectance from the water surfaces in both the VV and VH bands corresponds to a negative definite covariance matrix difference (green pixels, center left), the rapid receding of flood waters to a positive definite change (red pixels, center right)._

<img src="https://github.com/mortcanty/eesarseq/blob/main/screenshots/buzi2019.png?raw=1"><img/>  

Fig. 4 Buzi district, Mozambique: Six change maps of 15 in all for 16 images covering the period Jan 1, 2019, through June 7, 2019. Each image has an area of approximately 3000 km$^2$. The gray scale background is the temporal average of the VV band of all 16 images. The maps, read top to bottom, left to tight, are for the intervals April 18-March 2, March 2-14, March 14-20, March 20- 26, March 26-April 1 and April 1-7. Positive definite changes are red, negative definite green and indefinite yellow.

You can easily reproduce the above sequence with the interactive interface, remembering that negative definite changes are now shown as cyan.
The change signal from within the parially inundated city of Beira, just to the east of the scene, is considerably less convincing, as you can also verify yourself. This is due to the predominace of so-called double bounce scattering from built up areas, an effect which is present in both dry and flooded areas. 

### Crops

<Crop dynamics ...>

### Logging
<Amazon, Vancouver Island ...>